In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

REQUIRED_FILES = {
    'price.csv': ['Timestamp', 'Market_Price_EUR'],
    'demand.csv': ['Timestamp', 'Load_Demand_MW'],
    'solar.csv': ['Timestamp', 'Solar_Gen_MW'],
    'wind.csv': ['Timestamp', 'Wind_Gen_MW'],
    'nuclear.csv': ['Timestamp', 'Nuclear_Gen_MW'],
    'gas.csv': ['Timestamp', 'Gas_Price_EUR'],
    'coal.csv': ['Timestamp', 'Coal_Price_EUR'],
    'oil.csv': ['Timestamp', 'Oil_Price_EUR'],
    'fuelcell.csv': ['Timestamp', 'FuelCell_Gen_MW'],
    'ocean.csv': ['Timestamp', 'Ocean_Gen_MW'],
    'geothermal.csv': ['Timestamp', 'GeoThermal_Gen_MW'],
    'temp.csv': ['Timestamp', 'Ambient_Temp']
}

In [2]:
# 2. BESS AND STOCHASTIC OPTIMIZATION CONSTANTS ---

BESS_CAPACITY = 50.0   # MWh
MAX_C_RATE = 0.5       # 0.5C rate (25 MW)
RT_AFRR_PRICE = 250.0  # €/MWh for ancillary services revenue
DA_ARBITRAGE_THRESHOLD = 0.1 # 10% deviation for arbitrage decision
HOURS_TO_FORECAST = 24 # The length of the prediction window
NUM_SCENARIOS = 100    # Number of price paths for stochastic modeling
CVAR_ALPHA = 0.05      # 5% for P99/CVaR calculation
LAMBDA = 0.0001        # Risk Aversion Parameter (tunes P99 strategy)

In [3]:
# --- 3. CORE FUNCTIONS (Data Ingestion and Cleaning) ---

def create_and_load_data():
    """
    SIMULATES the upload process by creating dummy CSV files and reading them back.
    For real use, this function would be replaced by user file I/O.
    """
    print("\n--- 1. MARKET DATA INGESTION SIMULATION ---")
    print("Please ensure your files are named correctly and saved in the same folder.")
    print("Files created/loaded (MUST contain a 'Timestamp' column):\n")
    
    data_frames = {}
    total_hours = 90 * 24 # 90 days of hourly data

    for filename, cols in REQUIRED_FILES.items():
        df_data = {}
        dates = pd.date_range(start='2024-01-01', periods=total_hours, freq='h')
        df_data[cols[0]] = dates

        if 'Price' in cols[1] or 'Price' in filename:
            # Price/Cost data (Highly Volatile)
            base = 50 + 50 * np.sin(dates.dayofyear * 2 * np.pi / 365)
            df_data[cols[1]] = np.maximum(5, base + np.random.normal(0, 15, total_hours))
            
            # Introduce missing data for cleaning demonstration
            df_data[cols[1]].iloc[100:105] = np.nan 

        elif 'Gen_MW' in cols[1]:
            # Generation data (Cyclical/Seasonal)
            if 'Solar' in cols[1]:
                hourly = 100 * np.maximum(0, np.sin(dates.hour * np.pi / 12)) 
                seasonal = 0.5 + 0.5 * np.sin(dates.dayofyear * 2 * np.pi / 365)
                df_data[cols[1]] = np.maximum(0, hourly * seasonal + np.random.normal(0, 5, total_hours))
            else: # Wind/Nuclear/Other base gen
                df_data[cols[1]] = 50 + 20 * np.random.rand(total_hours)
                
        elif 'Load' in cols[1]:
            # Demand data (Hourly/Daily peaks)
            df_data[cols[1]] = 500 + 300 * np.maximum(0, np.sin(dates.hour * np.pi / 12)) + np.random.normal(0, 50, total_hours)
        
        elif 'Temp' in cols[1]:
            # Temperature data
            df_data[cols[1]] = 10 + 15 * np.sin(dates.dayofyear * 2 * np.pi / 365) + np.random.normal(0, 2, total_hours)

        df = pd.DataFrame(df_data).set_index(cols[0])
        # df.to_csv(filename) # Uncomment to actually save files
        data_frames[filename] = df
        print(f"   - {filename}: Loaded column '{cols[1]}'")
        
    return data_frames

def load_and_clean_data(data_frames):
    """Combines all data, handles missing values (FFILL), and performs feature engineering."""
    
    print("\n--- 2. DATA RELIABILITY & FEATURE ENGINEERING ---")
    
    # 1. Merge all DataFrames on the Index (Timestamp)
    master_df = pd.DataFrame(index=data_frames[list(data_frames.keys())[0]].index)
    
    for df in data_frames.values():
        master_df = master_df.merge(df, left_index=True, right_index=True, how='outer')

    # 2. Data Cleaning: Forward-Fill (FFILL) Missing Values
    print(f"   - Initial Missing Count (before cleaning): {master_df.isnull().sum().sum()}")
    master_df.ffill(inplace=True)
    
    # Drop any remaining NaNs (e.g., missing data at the very start of the series)
    master_df.dropna(inplace=True)
    print(f"   - Final Missing Count (after FFILL): {master_df.isnull().sum().sum()}")

    # 3. Feature Engineering
    master_df['Hour_of_Day'] = master_df.index.hour
    master_df['Day_of_Week'] = master_df.index.dayofweek
    
    # Consolidated Net Load Calculation (Crucial feature for forecasting)
    gen_cols = [c for c in master_df.columns if 'Gen_MW' in c]
    master_df['Total_Gen_MW'] = master_df[gen_cols].sum(axis=1)
    
    master_df['Net_Load_MW'] = master_df['Load_Demand_MW'] - master_df['Total_Gen_MW']
    
    # Isolate relevant columns for the final analysis (for the last 90 days)
    analysis_df = master_df.iloc[-HOURS_TO_FORECAST * 90:].copy()
    print(f"   - Final Dataset Size: {len(analysis_df)} hourly records.")
    
    return analysis_df

In [4]:
# --- 4. CORE FUNCTIONS (Forecasting and Optimization) ---

def run_forecasting(df):
    """Trains a simple Linear Regression model on multivariate data and forecasts the next 24h."""
    
    # Prepare features and target (Focus on 3 main drivers of Net Load)
    features = ['Net_Load_MW', 'Ambient_Temp', 'Hour_of_Day']
    target = 'Market_Price_EUR'
    
    # Handle the 'Ambient_Temp' column, which might be missing in simulation
    if 'Ambient_Temp' not in df.columns:
        df['Ambient_Temp'] = df.index.hour.map(lambda x: 15 + 10 * np.sin(x * np.pi / 12))
        features = ['Net_Load_MW', 'Hour_of_Day']

    X = df[features].values
    y = df[target].values

    # Scale data for better stability (using MinMaxScaler)
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    
    # Train the Model
    model = LinearRegression()
    model.fit(X_scaled, y)
    
    # Create the X_forecast set (assuming next 24 hours follow the average Net Load cycle)
    last_load = df['Net_Load_MW'].iloc[-1]
    
    X_forecast = []
    for h in range(HOURS_TO_FORECAST):
        # Simplistic forward projection: Last load + hourly cycle adjustment
        load_proj = last_load + 50 * np.sin(2 * np.pi * (h + 12) / 24)
        temp_proj = df['Ambient_Temp'].iloc[-24:].mean() # Use average temp
        hour_proj = (df.index[-1].hour + h + 1) % 24
        
        if 'Ambient_Temp' in features:
            X_forecast.append([load_proj, temp_proj, hour_proj])
        else:
            X_forecast.append([load_proj, hour_proj])

    # Scale the forecast inputs using the *trained* scaler
    X_forecast_scaled = scaler_X.transform(np.array(X_forecast))
    
    # Predict the next 24 prices
    forecast_prices = np.maximum(5.0, model.predict(X_forecast_scaled))
    
    return forecast_prices, df[target].iloc[-HOURS_TO_FORECAST:].values

def run_bess_stochastic_optimization(base_prices):
    """
    Implements the core stochastic optimization for P50 (Risk-Neutral) and P99 (Risk-Averse)
    using the Expected Profit and CVaR metrics, respectively.
    """
    
    def calculate_cvar(profits):
        """Calculates CVaR (Average of the worst CVAR_ALPHA profits)."""
        profits_sorted = np.sort(profits)
        tail_size = max(1, int(len(profits) * CVAR_ALPHA))
        return profits_sorted[:tail_size].mean()

    def generate_scenarios(prices):
        """Generates RT price paths based on base prices and a volatility factor."""
        scenarios = []
        NE_DRIVER = prices.mean() * 0.1
        for s in range(NUM_SCENARIOS):
            scenario_prices = []
            for h in range(HOURS_TO_FORECAST):
                ne_error = np.random.normal(0, 1)
                rt_volatility = NE_DRIVER * ne_error * 0.25 
                price = prices[h] + rt_volatility
                scenario_prices.append(np.maximum(5, price))
            scenarios.append(scenario_prices)
        return scenarios

    def evaluate_profit(scenario_prices, capacity_reserved):
        """Calculates total profit for one scenario."""
        capacity_arb = BESS_CAPACITY - capacity_reserved
        P_arb_max = MAX_C_RATE * capacity_arb 
        soc_arb = capacity_arb * 0.5
        total_arb_profit = 0
        
        avg_price = np.mean(scenario_prices)
        
        for price_rt in scenario_prices:
            price_da = avg_price # Simplified assumption: DA price is the average price
            
            # Dispatch decision based on DA threshold
            if price_da < avg_price * (1 - DA_ARBITRAGE_THRESHOLD) and soc_arb < capacity_arb:
                charge_power = min(capacity_arb - soc_arb, P_arb_max)
                total_arb_profit -= charge_power * price_rt * 0.95 # Cost realized at RT
                soc_arb += charge_power * 0.95 
            elif price_da > avg_price * (1 + DA_ARBITRAGE_THRESHOLD) and soc_arb > 0:
                discharge_power = min(soc_arb, P_arb_max)
                total_arb_profit += discharge_power * price_rt * 0.95 # Revenue realized at RT
                soc_arb -= discharge_power
        
        total_rt_profit = capacity_reserved * RT_AFRR_PRICE * HOURS_TO_FORECAST
        return total_arb_profit + total_rt_profit

    # Generate Scenarios
    scenarios = generate_scenarios(base_prices)
    RESERVATION_STEPS = np.linspace(0, BESS_CAPACITY, 11)
    
    results = []
    
    # Optimization Loop
    for capacity_reserved in RESERVATION_STEPS:
        profits = np.array([evaluate_profit(s, capacity_reserved) for s in scenarios])
        
        expected_profit = np.mean(profits)
        cvar = calculate_cvar(profits)
        risk_adjusted_profit = expected_profit - LAMBDA * abs(cvar) # Risk Mitigation Strategy

        results.append({
            'Reserved_MWh': capacity_reserved,
            'Expected_Profit': expected_profit,
            'CVaR_Profit': cvar,
            'Risk_Adjusted_Profit': risk_adjusted_profit
        })
        
    optimization_df = pd.DataFrame(results)

    # Find optimal points (P50 and P99)
    optimal_p50 = optimization_df.loc[optimization_df['Expected_Profit'].idxmax()]
    optimal_p99 = optimization_df.loc[optimization_df['Risk_Adjusted_Profit'].idxmax()]

    return optimization_df, optimal_p50, optimal_p99

In [5]:
# --- 5. MAIN EXECUTION AND REPORTING ---

def main_analysis():    
    # Step 1: Simulate File Upload and Load Data
    data_frames = create_and_load_data()
    
    # Step 2: Clean and Engineer Features
    master_df = load_and_clean_data(data_frames)
    
    # Step 3: Forecasting
    print("\n--- 3. PRICE FORECASTING ---")
    forecast_prices, actual_prices = run_forecasting(master_df)
    print(f"   - Next {HOURS_TO_FORECAST} hour prices forecasted.")
    
    # Step 4: BESS Stochastic Optimization (P50/P99)
    print("\n--- 4. BESS STOCHASTIC OPTIMIZATION ---")
    optimization_df, optimal_p50, optimal_p99 = run_bess_stochastic_optimization(forecast_prices)
    print(f"   - Optimal Risk-Neutral (P50) Reserve: {optimal_p50['Reserved_MWh']:.2f} MWh")
    print(f"   - Optimal Risk-Averse (P99) Reserve: {optimal_p99['Reserved_MWh']:.2f} MWh")
    
    # Step 5: Reporting and Graphing
    
    # Summary Metrics
    print("\n--- 5. PROFITABILITY AND STRATEGY SUMMARY ---")
    print(f"Risk-Neutral (P50) Strategy (Max E[Profit]):")
    print(f"   Reserved for aFRR: {optimal_p50['Reserved_MWh']:.1f} MWh | E[Profit]: {optimal_p50['Expected_Profit']:,.0f} €")
    print(f"Risk-Averse (P99) Strategy (Max Risk-Adj. Profit):")
    print(f"   Reserved for aFRR: {optimal_p99['Reserved_MWh']:.1f} MWh | E[Profit]: {optimal_p99['Expected_Profit']:,.0f} €")
    print(f"   CVaR (Worst 5% Average Profit): {optimal_p99['CVaR_Profit']:,.0f} €")
    
    generate_all_graphs(master_df, actual_prices, forecast_prices, optimization_df)
    
def generate_all_graphs(df, actual, forecast, opt_df):
    """Generates all required visualizations."""
    
    print("\n--- 6. GRAPH GENERATION ---")
    
    # --- Graph 1: Actual vs. Predicted Energy Prices ---
    plt.figure(figsize=(12, 6))
    
    full_series = np.concatenate([actual, forecast])
    actual_labels = [f'H{i}' for i in range(HOURS_TO_FORECAST)]
    forecast_labels = [f'F{i}' for i in range(HOURS_TO_FORECAST)]
    
    plt.plot(actual_labels, actual, label='Actual Historical Price', marker='o', color='blue', linestyle='-')
    plt.plot(forecast_labels, forecast, label='Predicted Price (Next 24h)', marker='x', color='red', linestyle='--')
    plt.axvline(x='F0', color='gray', linestyle=':', linewidth=2, label='Forecast Start')
    
    plt.title('Actual vs. Predicted Energy Prices (Forecasting Reliability)')
    plt.xlabel('Time (Hours)')
    plt.ylabel('Market Price (€/MWh)')
    plt.legend()
    plt.grid(True)
    plt.savefig('graph_1_price_forecast.png')
    
    # --- Graph 2: Optimization Decision (P50/P99 Frontier) ---
    plt.figure(figsize=(10, 6))
    plt.plot(opt_df['Reserved_MWh'], opt_df['Expected_Profit'], marker='o', label='Expected Profit (P50)', color='green')
    plt.plot(opt_df['Reserved_MWh'], opt_df['CVaR_Profit'], marker='s', label='CVaR Profit (P99 - Worst Case)', color='red')
    
    # Mark Optimal P50 Point
    opt_p50 = opt_df.loc[opt_df['Expected_Profit'].idxmax()]
    plt.axvline(opt_p50['Reserved_MWh'], color='green', linestyle='--', alpha=0.5, label=f"Optimal P50 Reserve ({opt_p50['Reserved_MWh']:.1f} MWh)")
    
    # Mark Optimal P99 Point
    opt_p99 = opt_df.loc[opt_df['Risk_Adjusted_Profit'].idxmax()]
    plt.axvline(opt_p99['Reserved_MWh'], color='red', linestyle='--', alpha=0.5, label=f"Optimal P99 Reserve ({opt_p99['Reserved_MWh']:.1f} MWh)")
    
    plt.title('Optimal Capacity Reservation Schedule (P50 vs P99)')
    plt.xlabel('Capacity Reserved for RT/Flexibility (MWh)')
    plt.ylabel('Profit (€)')
    plt.legend()
    plt.grid(True)
    plt.savefig('graph_2_bess_decision.png')
    
    # --- Graph 3: Correlation Heatmap (Market Dependencies) ---
    plt.figure(figsize=(10, 8))
    
    corr_cols = ['Market_Price_EUR', 'Net_Load_MW', 'Solar_Gen_MW', 'Gas_Price_EUR', 'Load_Demand_MW']
    corr_df = df[corr_cols].corr()
    
    sns.heatmap(corr_df, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Correlation Heatmap (Market Dependencies)')
    plt.tight_layout()
    plt.savefig('graph_3_correlation_heatmap.png')
    
    # --- Graph 4: Total Generation Mix Breakdown ---
    plt.figure(figsize=(10, 6))
    gen_cols = [c for c in df.columns if 'Gen_MW' in c and c != 'Total_Gen_MW']
    gen_mix = df[gen_cols].mean()
    
    plt.pie(gen_mix, labels=gen_mix.index, autopct='%1.1f%%', startangle=90, colors=plt.cm.Spectral(np.linspace(0, 1, len(gen_mix))))
    plt.title('Average Generation Mix Breakdown')
    plt.tight_layout()
    plt.savefig('graph_4_generation_mix.png')
    
    # --- Graph 5: Load vs. Net Load Profiles (Flexibility Need) ---
    plt.figure(figsize=(12, 6))
    
    # Use the last 7 days for clear visualization
    last_week = df.iloc[-24*7:] 
    
    plt.plot(last_week['Load_Demand_MW'].values, label='Gross Load Demand', color='orange', alpha=0.7)
    plt.plot(last_week['Net_Load_MW'].values, label='Net Load (Demand - Renewables)', color='blue')
    
    plt.title('Load Profiles vs. Net Load (Flexibility Need)')
    plt.xlabel('Hour Index (Last 7 Days)')
    plt.ylabel('Power (MW)')
    plt.legend()
    plt.grid(True)
    plt.savefig('graph_5_net_load_profile.png')

# Execute the main analysis
if __name__ == '__main__':
    main_analysis()


--- 1. MARKET DATA INGESTION SIMULATION ---
Please ensure your files are named correctly and saved in the same folder.
Files created/loaded (MUST contain a 'Timestamp' column):



AttributeError: 'Index' object has no attribute 'iloc'